In [ ]:
Intro:
no python code
Probabilistic counting and loglog
Hyperloglog++ explanation 
and python code
References. 



## Getting to HyperLogLog

### Introduction: what problem are we solvings?

HyperLogLog is an algorithm for approximating cardinality, a mathematical term for the number of unique elements in a group.  We ask questions about cardinality, sometimes called the COUNT DISTINCT problem, all the time. Maybe you care about the number of unique comments submitted to the FCC in [defense of net neutrality](https://arstechnica.com/tech-policy/2017/08/isp-funded-study-finds-huge-support-for-keeping-current-net-neutrality-rules/), or you wanted to know how many unique words were present in the first stanza of Jabberwocky, or maybe you’re just curious about the number of unique visitors to your boutique puppy hair salon. 

The simplest way to do this is to use a data structure with quick membership checks, like a hash table. Check if every entry in your dataset is in the dictionary; whenever it’s not, insert that element and increment your counter. 



In [27]:
jabber_string = ['twas','brillig','and','the','slithy','toves','did','gyre',
'and','gimble','in','the','wabe','all','mimsy','were','the','borogoves','and',
'the','mome','raths','outgrabe']

print(f"We have {len(jabber_string)} elements in our original dataset ")

holder_dictionary = dict()
distinct_count = 0
for element in jabber_string:
    if element not in holder_dictionary:   ##All-important membership check
        distinct_count+=1                  ##deeply un-pythonic way of doing this. 
        holder_dictionary[element]='present'
    else:
        pass

print(f"We have {distinct_count} distinct elements in this dataset.")

We have 23 elements in our original dataset 
We have 18 distinct elements in this dataset.



This is all well and good for a dataset with 23 elements. But if you have billions of visitors to your doggy spa page, this calculation becomes memory-hungry and computationally expensive. You could distribute the work to many worker nodes, but then you have distributed computing problems: you have to reconcile the results of each node’s computations and there’s a lot of data to pass back from each node. Using a map-reduce approach, even if you reduce locally on your worker nodes, you’ll still have to send back to the master node an array of values present in your reducer. 

An important question in solving this problem is: do you need to know _exactly_ how many visitors came to your site? If we are freed from the tyranny of the exact, our lives are much simpler. If we can put up with a close approximation of the number of distinct elements, we can get substantial memory and computational gains using algorithms from the field of Probabilistic Counting. 



## A Quick History of Probabilistic Counting.

#### A formal origin
For anyone with an inconvenient amount of data, approximate inference is a fairly common tool; you can stomach a reduction in accuracy for a reduction in execution time or computer memory required. At a large enough scale scale, exact inference on some problems would take millions of years to complete. But the application of probabilistic methods to *counting itself* is a relatively recent phenomenon. In 1978, Robert Morris at Bell Labs introduced a method for producing an estimated count from dataset.  Motivated by how the number of heads found by flipping an unbiased coin would produce roughly half the total number of coin flips, his method provided a way to specify the amount of error allowed for a given computational memory budget. This idea had clear industrial benefits, especially in memory-constrained applications like embedded systems. 

#### Philippe Flajolet
The idea of approximate counting appears to have jammed itself into the mind of French Computer Scientist/Mathematician Philippe Flajolet, who in 1985 wrote [a more extended treatment](http://algo.inria.fr/flajolet/Publications/Flajolet85c.pdf) of Morris’ approximate counting algorithm. Soon afterward, he published a paper [introducing a probabilistic method](http://algo.inria.fr/flajolet/Publications/FlMa85.pdf) for the count distinct problem. 


In his algorithm, Probabilistic Counting with Stochastic Averaging (PCSA), he used the idea that  based on the idea that, given a uniform hash function, the position of the leftmost zero in the hash of the object to be counted is an approximation of logsub2 of n. Calling that R, he scaled lambda(n) = (1/phi)2^^Rsubn. The approximation was too inexact, so he proposed using stochastic averaging on the R’s. While hashing, group the records into M groups by using hash(input) mod(M), then calculating the Rsubn for that group. Finally, aggregate.




In [ ]:
max_lsb = -1
ones_found = ["0" for i in range(128)]
for element in jabber_string:
    m = md5()
    m.update(element.encode())
    hashcode = m.hexdigest()
    asint = int(hashcode,16)
    #binstring = format(asint,'0128b')
    #print(binstring)
    current_lsb = least_significant_bit(asint)
    ones_found[current_lsb]="1"
    max_lsb = max(max_lsb,current_lsb)

In [149]:
from hashlib import md5 

def least_significant_bit(input_int):
    """
    Return the least significant bit from an integer
    With thanks to Mark Dickinson on SO: https://stackoverflow.com/questions/34166566/whats-the-fastest-method-to-return-the-position-of-the-least-significant-bit-se
    """
    return  (input_int & -input_int).bit_length() - 1

def hash_and_binarize(input_string):
    """
    This takes our input strings and returns a string of ones and zeros for calculating the least significant bit
    """
    m = md5()
    m.update(input_string.encode())
    hashcode = m.hexdigest()
    asint = int(hashcode,16)
    return asint

max_lsb = -1
ones_found = ["0" for i in range(128)]
for element in jabber_string:
    asint = hash_and_binarize(element)
    #binstring = format(asint,'0128b')
    #print(binstring)
    current_lsb = least_significant_bit(asint)
    ones_found[current_lsb]="1"
    max_lsb = max(max_lsb,current_lsb)
    
r_estimate = ones_found.index("0")

φ =0.77351
λ = (1/φ)*2**r_estimate

print(f"the leftmost zero in our ones-place-holder for this dataset was {r_estimate}, so our estimate of the cardinality is  {int(λ)} ")


the leftmost zero in our ones-place-holder for this dataset was 5, so our estimate of the cardinality is  41 


But this was not an accurate enough measure in general, so he used the hash function to split the dataset up. 

In [170]:
from hashlib import md5 
import math
import numpy as np

def least_significant_bit(input_int):
    """
    Return the least significant bit from an integer
    With thanks to Mark Dickinson on SO: https://stackoverflow.com/questions/34166566/whats-the-fastest-method-to-return-the-position-of-the-least-significant-bit-se
    """
    return  (input_int & -input_int).bit_length() - 1

def group_and_int(input_string, num_groups):
    """
    This takes our input strings and returns a string of ones and zeros for calculating the least significant bit
    """
    m = md5()
    m.update(input_string.encode())
    hashcode = m.hexdigest()
    asint = int(hashcode,16)
    alpha = asint % num_groups 
    rem_int = math.floor(asint/num_groups)
    return (alpha, rem_int)

num_batches = 5
max_lsb = -1
ones_found = ["0" for i in range(128)]
group_dict = {}
for element in jabber_string:
    group, remaining_int = group_and_int(element, num_groups=num_batches)
    print(group, remaining_int)
    if group not in group_dict.keys():
        group_dict[group] = [remaining_int]
    else:
        group_dict[group].append(remaining_int)
    #current_lsb = least_significant_bit(asint)
    #ones_found[current_lsb]="1"
    #max_lsb = max(max_lsb,current_lsb)
    

R_samples = []
for group in group_dict.keys():
    current_bitholder = ["0" for _ in range(128)]
    for bin_string in group_dict[group]:
        current_lsb = least_significant_bit(bin_string)
        print(current_lsb)
        current_bitholder[current_lsb]="1"
    sample_R = current_bitholder.index("0")
    print(sample_R)
    R_samples.append(sample_R)


R_estimate = (sum(R_samples))/num_batches
φ =0.77351
λ = (1/φ)*2**R_estimate

print(f"the leftmost zero in our ones-place-holder for this dataset was {R_estimate}, so our estimate of the cardinality is  {int(λ)} ")


2 64062270616557689651147354933901131776
3 52466348488662174541902960035912220672
4 50607618690934308376383652158867767296
4 38219638942521735058138015913826320384
0 2982281716540308022146558625184743424
1 64974960805370520535659059556500111360
2 63410106522289915614136348768353648640
2 57437846345176776457867036706757148672
4 50607618690934308376383652158867767296
2 53999122111702458886292074517032337408
4 5239806017226476968630914916380835840
4 38219638942521735058138015913826320384
2 8349032152281506580856159421811654656
4 42935776154101823344836562169075073024
3 29617179120478454431849966109047390208
4 11432843670683485205717818295235641344
4 38219638942521735058138015913826320384
1 43576542887033675643318322065779458048
4 50607618690934308376383652158867767296
4 38219638942521735058138015913826320384
0 39920144810055736402360145151554224128
2 50188371228337761670735655307717902336
0 48039373802239570916549705365184315392
74
73
73
75
70
73
0
73
74
0
75
73
75
76
73
77
71
73
75
73
0


In [166]:

R_samples = []
for group in group_dict.keys():
    current_bitholder = ["0" for _ in range(128)]
    for bin_string in group_dict[group]:
        current_lsb = least_significant_bit(bin_string)
        print(current_lsb)
        current_bitholder[current_lsb]="1"
    sample_R = current_bitholder.index("0")
    print(sample_R)
    R_samples.append(sample_R)



74
73
73
75
70
73
0
73
74
0
75
73
75
76
73
77
71
73
75
73
0
70
72
73
0
73
74
0


##Notes

In [121]:
https://en.wikipedia.org/wiki/Flajolet–Martin_algorithm


SyntaxError: invalid syntax (<ipython-input-121-58ea362ec007>, line 1)


Linear Counting?

He also wrote Loglog (http://algo.inria.fr/flajolet/Publications/DuFl03-LNCS.pdf) (and superLogLog)

Super Loglog is loglog + Truncation + Restriction

Hyperloglog http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf Hyperloglog’s major innovation was switching from an arithmetic mean (v1 + v2 + v3)/n to a 

Google carried the torch. https://research.google.com/pubs/pub40671.html

Finally, Hyperloglog TailCut and Tailcut+http://cse.seu.edu.cn/PersonalPage/csqjxiao/csqjxiao_files/papers/INFOCOM17-slides.pdf



of the true sing a relatively smaller amount of memory to store a value that is only incremented probabilistically. The paper 

https://www.inf.ed.ac.uk/teaching/courses/exc/reading/morris.pdf

Educational Resources:
https://en.wikipedia.org/wiki/HyperLogLog
http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf
http://druid.io/blog/2014/02/18/hyperloglog-optimizations-for-real-world-systems.html 


## HyperLogLog:

Hash the data and set the count the maximum number of leading zeroes as m. The estimate for the number of unique elements is 2*m. 

To reduce the variance of this estimator, they split the data into multiple sets, estimate the cardinality for each subset, and use the harmonic mean to come up with the hyperloglog estimate.


 In general, a COUNT (DISTINCT URL) requires memory space proportional to the number of unique values. HyperLogLog requires constant memory, runs on a stream (so you can add to it) over time without needing to record the actual data.

